In [40]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path)

import datetime as dt

import pandas as pd
import numpy as np


## Cursory check of LLM capacity to detect distortions

In [2]:
from langchain_ollama.chat_models import ChatOllama
default_chat_model = "llama3.2"

llm = ChatOllama(model=default_chat_model, base_url="http://localhost:11434")  # Explicitly set base_url

In [3]:
def evaluator_llm(text,threshold=0):
    query = f"Be concise. Does this text have cognitive distortions in it \"{text}\"?"
    r = llm.invoke(query).content
    return r.lower().startswith("yes")


In [4]:
texts = [
    "I am such a failure I never do anything right.",
    "I am a software developer doing coding.",
    "I am a man sitting on the chair behind the table.",
    "There is a chair behind the table."
]
for r in texts:
    q = f"Be concise. Does this text have cognitive distortions in it \"{r}\"?"
    d = evaluator_llm(r)
    print(f"{d}: {q}")
    
    

True: Be concise. Does this text have cognitive distortions in it "I am such a failure I never do anything right."?
True: Be concise. Does this text have cognitive distortions in it "I am a software developer doing coding."?
True: Be concise. Does this text have cognitive distortions in it "I am a man sitting on the chair behind the table."?
False: Be concise. Does this text have cognitive distortions in it "There is a chair behind the table."?


## Cursory compare different LLMs

In [39]:
texts = [
    "I am such a failure I never do anything right.",
    "I am a software developer doing coding.",
    "I am a man sitting on the chair behind the table.",
    "There is a chair behind the table."
]
llm_llama32 = ChatOllama(model="llama3.2", base_url="http://localhost:11434")  # Explicitly set base_url
def evaluator_llm_llama32(text,threshold=0):
    query = f"Be concise. Does this text have cognitive distortions in it \"{text}\"?"
    r = llm_llama32.invoke(query).content
    return r.lower().startswith("yes"), r
for r in texts:
    q = f"Be concise. Does this text have cognitive distortions in it \"{r}\"?"
    d, r = evaluator_llm_llama32(r)
    print(f"{d}: {q} ==> {r}")

True: Be concise. Does this text have cognitive distortions in it "I am such a failure I never do anything right."? ==> Yes, the text contains cognitive distortions. The phrase "never do anything right" is an absolute statement and not supported by evidence, which is a characteristic of all-or-nothing thinking (black-and-white thinking).
True: Be concise. Does this text have cognitive distortions in it "I am a software developer doing coding."? ==> Yes, there is an overgeneralization in the sentence. It implies that being a software developer means only coding, which isn't necessarily true. A developer may also work on project management, testing, design, etc.
True: Be concise. Does this text have cognitive distortions in it "I am a man sitting on the chair behind the table."? ==> Yes, it has two common cognitive distortions:

1. Euphemistic self-deception (downplaying one's true identity): The statement is a polite way of saying "I'm a male sitting on a chair."
2. Self-referential err

In [38]:
texts = [
    "I am such a failure I never do anything right.",
    "I am a software developer doing coding.",
    "I am a man sitting on the chair behind the table.",
    "There is a chair behind the table."
]
llm_qwen2 = ChatOllama(model="qwen2", base_url="http://localhost:11434")  # Explicitly set base_url
def evaluator_llm_qwen2(text,threshold=0):
    query = f"Be concise. Does this text have cognitive distortions in it \"{text}\"?"
    r = llm_qwen2.invoke(query).content
    return r.lower().startswith("yes"), r
for r in texts:
    q = f"Be concise. Does this text have cognitive distortions in it \"{r}\"?"
    d, r = evaluator_llm_qwen2(r)
    print(f"{d}: {q} ==> {r}")

True: Be concise. Does this text have cognitive distortions in it "I am such a failure I never do anything right."? ==> Yes, the text contains a cognitive distortion known as all-or-nothing thinking, which is characterized by seeing things in black-and-white categories, with no shades of gray. In this case, the statement suggests that there is absolute failure without considering any instances where the individual might have succeeded or at least tried successfully.
False: Be concise. Does this text have cognitive distortions in it "I am a software developer doing coding."? ==> No, the statement "I am a software developer doing coding" does not contain any obvious cognitive distortions. It is straightforward and directly states someone's profession and activity. Cognitive distortions are typically irrational or exaggerated thought patterns that can lead to emotional distress, but this sentence doesn't exhibit those characteristics.
False: Be concise. Does this text have cognitive disto

## Explore performance of "our out of the box" model with dataset 1 (original binary)

In [21]:
binary_dataset_file_path = "./data/corpora/English/distortions/halilbabacan/raw_Cognitive_distortions.csv" 
df = pd.read_csv(binary_dataset_file_path)
df.insert(1, "N/A text", value = np.nan)
df.insert(3, "N/A label", value = np.nan)
df.head(10)


,Text,N/A text,Label,N/A label
0,I'm such a failure I never do anything right.,NaN,Distortion,NaN
1,Nobody likes me because I'm not interesting.,NaN,Distortion,NaN
2,I can't try new things because I'll just mess...,NaN,Distortion,NaN
3,My boss didn't say 'good morning' she must be...,NaN,Distortion,NaN
4,My friend didn't invite me to the party I mus...,NaN,Distortion,NaN
5,I didn't get the job so I must be incompetent.,NaN,Distortion,NaN
6,I'm always unlucky. Good things only happen t...,NaN,Distortion,NaN
7,Everyone thinks I'm stupid because I made a m...,NaN,Distortion,NaN
8,I'll never be successful because I failed my ...,NaN,Distortion,NaN
9,Nobody cares about me because they didn't ask...,NaN,Distortion,NaN


In [41]:
from pygents.aigents_api import TextMetrics

def language_metrics(lang,metrics_list):
    metrics = {}
    for m in metrics_list:
        metrics[m] = './data/dict/' + lang + '/' + m + '.txt'
    return metrics


distortion_labels = ['positive','negative','rude',
'catastrophizing','dichotomous-reasoning','disqualifying-positive','emotional-reasoning','fortune-telling',
'labeling','magnification','mental-filtering','mindreading','overgeneralizing','personalizing','should-statement']
tm = TextMetrics(language_metrics('en',distortion_labels),debug=False)

def f1_from_counts(true_positive, true_negative, false_positive, false_negative):
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return 2 * precision * recall / (precision + recall) if precision > 0 or recall > 0 else 0 

def evaluate_df(df,evaluator,threshold,debug=False):
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    for _, row in df.iterrows():
        # Text definition: first, check the 2nd column; if NaN, take the text from the 1st column.
        text = row.iloc[1] if pd.notna(row.iloc[1]) else row.iloc[0]
        primary_distortion = row.iloc[2]  # The main cognitive distortion from the 3rd column
        secondary_distortion = row.iloc[3] if pd.notna(row.iloc[3]) else None  # The secondary distortion from the 4th column, if it exists
        ground_distortion = False if primary_distortion == 'No Distortion' else True
                       
        our_distortion = evaluator(text,threshold)
        
        # https://en.wikipedia.org/wiki/F-score
        if ground_distortion == True and our_distortion == True:
            true_positive += 1
        if ground_distortion == False and our_distortion == True:
            false_positive += 1
        if ground_distortion == False and our_distortion == False:
            true_negative += 1
        if ground_distortion == True and our_distortion == False:
            false_negative += 1

        if debug:
            print(ground_distortion,our_distortion,text[:20],metrics)

    return f1_from_counts(true_positive, true_negative, false_positive, false_negative) 


def our_evaluator_any(text,threshold):
    metrics = tm.get_sentiment_words(text)
    for m in metrics:
        if metrics[m] > threshold:
            return True
    return False

def our_evaluator_avg(text,threshold):
    metrics = tm.get_sentiment_words(text)
    l = list(metrics.values())
    avg = sum(l) / len(l) if  len(l) > 0 else 0
    if avg > threshold:
        return True
    return False
  

In [23]:
for threshold in [0.0,0.01,0.05,0.1,0.2,0.4,0.6,0.8]:
    f1 = evaluate_df(df,our_evaluator_any,threshold)
    print(threshold, f1)

0.0 0.8443643512450851
0.01 0.8443643512450851
0.05 0.8443643512450851
0.1 0.8443643512450851
0.2 0.8445027035883992
0.4 0.8444669365721997
0.6 0.5569898379566054
0.8 0.013021830716200687


In [24]:
for threshold in [0.0,0.01,0.05,0.1,0.2,0.4,0.6,0.8]:
    f1 = evaluate_df(df,our_evaluator_avg,threshold)
    print(threshold, f1)

0.0 0.8443643512450851
0.01 0.8443643512450851
0.05 0.8443643512450851
0.1 0.8443643512450851
0.2 0.8447795443369939
0.4 0.886836935166994
0.6 0.18404478656403078
0.8 0.007680491551459293


## Explore performance of LLM (llama3.2) with dataset 1 (original binary)

In [9]:
df[:5]

,Text,N/A text,Label,N/A label
0,I'm such a failure I never do anything right.,NaN,Distortion,NaN
1,Nobody likes me because I'm not interesting.,NaN,Distortion,NaN
2,I can't try new things because I'll just mess...,NaN,Distortion,NaN
3,My boss didn't say 'good morning' she must be...,NaN,Distortion,NaN
4,My friend didn't invite me to the party I mus...,NaN,Distortion,NaN


In [20]:
t0 = dt.datetime.now()

f1 = evaluate_df(df,evaluator_llm,0,debug=False)

t1 = dt.datetime.now()
delta = t1 - t0
print(f1,delta.total_seconds(),delta.total_seconds()/len(df))


0.8565737051792828 3474.569199 0.9851344482563085


## Explore performance of LLM (qwen2) with dataset 1 (original binary)

In [26]:
llm_qwen2 = ChatOllama(model="qwen2", base_url="http://localhost:11434")  # Explicitly set base_url
def evaluator_llm_qwen2(text,threshold=0):
    query = f"Be concise. Does this text have cognitive distortions in it \"{text}\"?"
    r = llm_qwen2.invoke(query).content
    return r.lower().startswith("yes")

t0 = dt.datetime.now()

f1 = evaluate_df(df,evaluator_llm_qwen2,0,debug=False)

t1 = dt.datetime.now()
delta = t1 - t0
print(f1,delta.total_seconds(),delta.total_seconds()/len(df))


0.8573050719152157 6282.644701 1.7812998868726964


## Explore performance of LLM (llama3.2 and qwen2) with dataset 3 (joint 1+2)


In [27]:
# Dataset: Unclassified distortions (halilbabacan)
# Paper: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4582307
# Data: https://huggingface.co/datasets/halilbabacan/autotrain-data-cognitive_distortions
# https://huggingface.co/datasets/halilbabacan/autotrain-data-cognitive_distortions/tree/main/raw
# https://huggingface.co/datasets/halilbabacan/autotrain-data-cognitive_distortions/blob/main/raw/Cognitive_distortions.csv
    
binary_dataset_file_path = "./data/corpora/English/distortions/halilbabacan/raw_Cognitive_distortions.csv" 

In [28]:
# Dataset: Multiple Distorions (sagarikashreevastava)
# Paper: https://aclanthology.org/2021.clpsych-1.17/
# Data: https://www.kaggle.com/datasets/sagarikashreevastava/cognitive-distortion-detetction-dataset

# !pip install kagglehub
import kagglehub
multiclass_dataset_path = kagglehub.dataset_download("sagarikashreevastava/cognitive-distortion-detetction-dataset")
print("Path to dataset files:", multiclass_dataset_path)
multiclass_dataset_file_path = multiclass_dataset_path + "/Annotated_data.csv"


Path to dataset files: C:\Users\anton\.cache\kagglehub\datasets\sagarikashreevastava\cognitive-distortion-detetction-dataset\versions\1


In [29]:
df1 = pd.read_csv(binary_dataset_file_path)
df1 = df1.rename(columns={'Text': 'Patient Question', 'Label': 'Dominant Distortion'})
df1.insert(1, "Distorted part", value = np.nan)
df1.insert(3, "Secondary Distortion (Optional)l", value = np.nan)
df1

,Patient Question,Distorted part,Dominant Distortion,Secondary Distortion (Optional)l
0,I'm such a failure I never do anything right.,NaN,Distortion,NaN
1,Nobody likes me because I'm not interesting.,NaN,Distortion,NaN
2,I can't try new things because I'll just mess...,NaN,Distortion,NaN
3,My boss didn't say 'good morning' she must be...,NaN,Distortion,NaN
4,My friend didn't invite me to the party I mus...,NaN,Distortion,NaN
...,...,...,...,...
3522,Since then whenever my mother is out alone I b...,NaN,Distortion,NaN
3523,My family hate him but they didn’t met him at ...,NaN,Distortion,NaN
3524,However I am not happy at the least only half ...,NaN,Distortion,NaN
3525,Now I am at university my peers around me all ...,NaN,Distortion,NaN


In [30]:
df2 = pd.read_csv(multiclass_dataset_file_path) 
df2 = df2.drop('Id_Number', axis=1) # delete columnb with id 
df2

,Patient Question,Distorted part,Dominant Distortion,Secondary Distortion (Optional)
0,"Hello, I have a beautiful,smart,outgoing and a...",The voice are always fimilar (someone she know...,Personalization,NaN
1,Since I was about 16 years old I’ve had these ...,I feel trapped inside my disgusting self and l...,Labeling,Emotional Reasoning
2,So I’ve been dating on and off this guy for a...,NaN,No Distortion,NaN
3,My parents got divorced in 2004. My mother has...,NaN,No Distortion,NaN
4,I don’t really know how to explain the situati...,I refused to go because I didn’t know if it wa...,Fortune-telling,Emotional Reasoning
...,...,...,...,...
2525,I’m a 21 year old female. I spent most of my l...,NaN,No Distortion,NaN
2526,I am 21 female and have not had any friends fo...,Now I am at university my peers around me all ...,Overgeneralization,NaN
2527,From the U.S.: My brother is 19 years old and ...,He claims he’s severely depressed and has outb...,Mental filter,Mind Reading
2528,From the U.S.: I am a 21 year old woman who ha...,NaN,No Distortion,NaN


In [31]:
df3 = pd.concat([df1, df2], ignore_index=True)
df3

,Patient Question,Distorted part,Dominant Distortion,Secondary Distortion (Optional)l,Secondary Distortion (Optional)
0,I'm such a failure I never do anything right.,NaN,Distortion,NaN,NaN
1,Nobody likes me because I'm not interesting.,NaN,Distortion,NaN,NaN
2,I can't try new things because I'll just mess...,NaN,Distortion,NaN,NaN
3,My boss didn't say 'good morning' she must be...,NaN,Distortion,NaN,NaN
4,My friend didn't invite me to the party I mus...,NaN,Distortion,NaN,NaN
...,...,...,...,...,...
6052,I’m a 21 year old female. I spent most of my l...,NaN,No Distortion,NaN,NaN
6053,I am 21 female and have not had any friends fo...,Now I am at university my peers around me all ...,Overgeneralization,NaN,NaN
6054,From the U.S.: My brother is 19 years old and ...,He claims he’s severely depressed and has outb...,Mental filter,NaN,Mind Reading
6055,From the U.S.: I am a 21 year old woman who ha...,NaN,No Distortion,NaN,NaN


In [43]:
llm_llama32 = ChatOllama(model="llama3.2", base_url="http://localhost:11434")  # Explicitly set base_url
def evaluator_llm_llama32(text,threshold=0):
    query = f"Be concise. Does this text have cognitive distortions in it \"{text}\"?"
    r = llm_llama32.invoke(query).content
    return r.lower().startswith("yes")

t0 = dt.datetime.now()

f1 = evaluate_df(df3,evaluator_llm_llama32,0,debug=False)

t1 = dt.datetime.now()
delta = t1 - t0
print(f1,delta.total_seconds(),delta.total_seconds()/len(df3))

0.8271139341008337 6202.05872 1.0239489384183589


In [45]:
llm_qwen2 = ChatOllama(model="qwen2", base_url="http://localhost:11434")  # Explicitly set base_url
def evaluator_llm_qwen2(text,threshold=0):
    query = f"Be concise. Does this text have cognitive distortions in it \"{text}\"?"
    r = llm_qwen2.invoke(query).content
    return r.lower().startswith("yes")

t0 = dt.datetime.now()

f1 = evaluate_df(df3,evaluator_llm_qwen2,0,debug=False)

t1 = dt.datetime.now()
delta = t1 - t0
print(f1,delta.total_seconds(),delta.total_seconds()/len(df3))

0.8087457952907255 11675.313834 1.9275736889549282
